In [146]:
import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
csv_file = pd.read_csv(f"{location}/ERC_correlation.csv")

csv_file.index = csv_file.columns
string_to_remove = "_nostop.fas.aligned.fas.best"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)

string_to_remove = ".fas.ClipKIT"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)


matrix_long = csv_file.reset_index().melt(
    id_vars="index",         # Keeps row names as a column
    var_name="Gene1",       # New name for columns
    value_name="Gene2"       # New name for matrix values
)

# Rename 'index' to 'Row' for clarity
diapause = matrix_long.rename(columns={"index": "Row"})

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"
# location = "/mnt/c/Users/Saurav Baral/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"{location}/correlation_2_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0][1:]
        anno_name = lines.split(",")[-1].replace('"', '')
        gene_anno_dic[gene_name]= anno_name.strip()


diapause_rn = diapause.rename(columns={"Gene2": "Correlation"})
diapause_rn = diapause_rn.rename(columns={"Gene1": "Gene2"})
diapause_rn = diapause_rn.rename(columns={"Row": "Gene1"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

diapause_rn["Annotation_gene1"] = diapause_rn["Gene1"].apply(check_annotation)
diapause_rn["Annotation_gene2"] = diapause_rn["Gene2"].apply(check_annotation)

# print(diapause_rn)

genes = diapause_rn["Gene1"].unique()
# df_filtered = diapause_rn[diapause_rn["Correlation"] >= 0.3]
df_filtered = diapause_rn
gene_dictionary = {}
to_remove = ["tim","9062at7088","per","TIMELESS","CLOCK"]
for row in df_filtered.itertuples():
    if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
        if row.Annotation_gene1 not in gene_dictionary:
            gene_dictionary[row.Annotation_gene1] = {}
        if row.Annotation_gene1 != row.Annotation_gene2:
            gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation




In [147]:
print(diapause_rn)

                Gene1       Gene2  Correlation Annotation_gene1  \
0          7556at7088  7556at7088          NaN            npr-9   
1         14105at7088  7556at7088     0.185247           ACOT13   
2          7117at7088  7556at7088     0.194968           NPEPL1   
3          8175at7088  7556at7088     0.138848            NR2C2   
4         12496at7088  7556at7088     0.032970             PIGF   
...               ...         ...          ...              ...   
20894036   7675at7088  4241at7088     0.107122            HDAC8   
20894037    360at7088  4241at7088     0.183250            KCNT2   
20894038  13468at7088  4241at7088    -0.037958      13468at7088   
20894039   3921at7088  4241at7088     0.345078            TUBG2   
20894040   4241at7088  4241at7088          NaN            SF3B2   

         Annotation_gene2  
0                   npr-9  
1                   npr-9  
2                   npr-9  
3                   npr-9  
4                   npr-9  
...                   ...  

In [157]:
gene_dict = gene_dictionary

import random
gene_dict = dict(random.sample(list(gene_dictionary.items()), 2000))
gene_dict["period"] = gene_dictionary["period"]
gene_dict["timeless"] = gene_dictionary["timeless"]
gene_dict["cry2"] = gene_dictionary["cry2"]
gene_dict["clock"] = gene_dictionary["clock"]
gene_dict["cycle"] = gene_dictionary["cycle"]
gene_dict["cwo"] = gene_dictionary["cwo"]
# "cry2","clock","cycle","cwo","cry1a","cry1b","timeout"
import networkx as nx

# Create a graph
G = nx.Graph()

# Add edges with similarity weights
for gene, neighbors in gene_dict.items():
    for neighbor, similarity in neighbors.items():
        G.add_edge(gene, neighbor, weight=similarity)


import numpy as np

# List of genes
genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))

# Fill the similarity matrix
for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = gene_dict[gene1][gene2]
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)

print("Similarity Matrix:")
print(similarity_matrix)

Similarity Matrix:
[[ 0.          0.17323963 -0.0486489  ...  0.21233962 -0.17105896
  -0.04268958]
 [ 0.17323963  0.          0.09402073 ...  0.1671138   0.18802953
  -0.13570554]
 [-0.0486489   0.09402073  0.         ...  0.05984326  0.09334588
   0.02647593]
 ...
 [ 0.21233962  0.1671138   0.05984326 ...  0.         -0.05286382
   0.30115042]
 [-0.17105896  0.18802953  0.09334588 ... -0.05286382  0.
  -0.11121358]
 [-0.04268958 -0.13570554  0.02647593 ...  0.30115042 -0.11121358
   0.        ]]


In [158]:
if "period" in genes:
    print(gene_dict["period"])

{'npr-9': 0.151246959909631, 'ACOT13': 0.0896936378592373, 'NPEPL1': 0.0551161506471083, 'NR2C2': 0.0003739022725638, 'PIGF': 0.021894918523445, 'AASS': -0.0505233556430518, 'LANCL1': -0.0060751980233622, 'THOC3': -0.0474046641730502, '13753at7088': 0.0378127036669959, 'RAPH1': 0.14353145040621, '11043at7088': 0.140904432460597, '6311at7088': -0.0050848120420668, 'PRRC2C': 0.111397181706434, '10291at7088': 0.066456022332915, 'MRPS31': 0.0647733109208979, 'GINS2': 0.162915636865756, 'NDUFB2': 0.0888753156729391, 'TENM2': 0.132073946624533, 'TPCN1': -0.0588127742115012, 'CHKA': 0.0197648046954698, '13904at7088': 0.116754251131664, '9785at7088': -0.0539108058796304, 'PPP2R4': 0.0065636419953695, '2842at7088': 0.147083368850152, 'GLG1': -0.0605622148587345, 'TRAPPC10': 0.0336989784431114, 'DEGS1': -0.0067690714514461, 'DDX17': -0.0470233573748171, '3528at7088': 0.0233598634868025, 'PIGK': 0.0231156508497101, 'SLBP': 0.0992344379011591, 'NFS1': 0.241825603121001, 'TRAPPC2': 0.09401588747232

In [71]:
def average_similarity(cluster, similarity_matrix, genes):
    """
    Compute the average similarity within a cluster.
    """
    if len(cluster) < 2:
        return 0.0  # No similarity for single-gene clusters
    
    # Get indices of genes in the cluster
    indices = [genes.index(gene) for gene in cluster]
    
    # Extract the submatrix for the cluster
    submatrix = similarity_matrix[np.ix_(indices, indices)]
    
    # Compute the average similarity (excluding diagonal)
    return np.sum(submatrix) / (len(cluster) * (len(cluster) - 1))


from scipy.stats import hmean

def average_similarity(cluster, similarity_matrix, genes):
    """
    Compute the harmonic mean of similarities within a cluster.
    """
    if len(cluster) < 2:
        return 0.0  # No similarity for single-gene clusters
    
    # Get indices of genes in the cluster
    indices = [genes.index(gene) for gene in cluster]
    
    # Extract the submatrix for the cluster
    submatrix = similarity_matrix[np.ix_(indices, indices)]
    
    # Flatten the submatrix and exclude diagonal (self-similarity)
    similarities = submatrix[np.triu_indices(len(cluster), k=1)]
    
    # Compute the harmonic mean of similarities
    if np.any(similarities == 0):
        return 0.0  # Harmonic mean is undefined if any similarity is 0
    return hmean(similarities)


def greedy_clustering(similarity_matrix, genes, max_clusters=2):
    """
    Greedy clustering algorithm to maximize average similarity within clusters.
    """
    clusters = []
    remaining_genes = set(genes)
    import datetime
    x = datetime.datetime.now()
    while remaining_genes:
        
        

        print("Cluster: ",len(clusters), (datetime.datetime.now() - x), datetime.datetime.now())
        # Start a new cluster with the most similar pair
        best_pair = None
        best_similarity = -1
        
        for i, gene1 in enumerate(remaining_genes):
            if i% 100 == 0:
                print(i, end = " ")
            for j, gene2 in enumerate(remaining_genes):
                if i >= j:
                    continue  # Avoid duplicate pairs and self-pairs
                sim = similarity_matrix[genes.index(gene1)][genes.index(gene2)]
                if sim > best_similarity:
                    best_similarity = sim
                    best_pair = (gene1, gene2)
        
        if not best_pair:
            break  # No more pairs to add
        
        # Initialize the cluster with the best pair
        cluster = set(best_pair)
        remaining_genes -= cluster
        
        # Add genes to the cluster if they increase average similarity
        
        for gene in list(remaining_genes):
            candidate_cluster = cluster | {gene}
            new_avg_sim = average_similarity(candidate_cluster, similarity_matrix, genes)
            # print(new_avg_sim)
            # if new_avg_sim > average_similarity(cluster, similarity_matrix, genes):
            if new_avg_sim > 0.40:
                cluster.add(gene)
                remaining_genes.remove(gene)
        if len(cluster) > 2:
            clusters.append(cluster)
        print(clusters, average_similarity(cluster, similarity_matrix, genes))
        if len(clusters) >= max_clusters:
            break  # Stop after reaching the maximum number of clusters
    
    return clusters

In [125]:
clusters = greedy_clustering(similarity_matrix, genes, max_clusters=20)
import datetime


# Print the clusters
for i, cluster in enumerate(clusters):
    print(f"Cluster {i}: {cluster}")
    if "period" in cluster:
        print(f"Cluster {i}: {cluster}")

Cluster:  0 0:00:00.000002 2025-01-28 11:41:29.538786
0 100 200 

KeyboardInterrupt: 

In [29]:
average_similarity({'timeless', 'cycle', 'clock', 'cry2', 'period'}, similarity_matrix, genes)

0.5102554569658607

In [164]:
import multiprocessing as mp

def average_similarity(cluster, similarity_matrix, genes):
    """
    Compute the average similarity within a cluster.
    """
    if len(cluster) < 2:
        return 0.0  # No similarity for single-gene clusters
    
    # Get indices of genes in the cluster
    indices = [genes.index(gene) for gene in cluster]
    
    # Extract the submatrix for the cluster
    submatrix = similarity_matrix[np.ix_(indices, indices)]
    
    # Compute the average similarity (excluding diagonal)
    return np.sum(submatrix) / (len(cluster) * (len(cluster) - 1))


def find_best_pair(genes, similarity_matrix, remaining_genes):
    """
    Helper function to find the most similar pair of genes in parallel.
    """
    best_pair = None
    best_similarity = -1
    
    for i, gene1 in enumerate(remaining_genes):
        for j, gene2 in enumerate(remaining_genes):
            if i >= j:
                continue  # Avoid duplicate pairs and self-pairs
            sim = similarity_matrix[genes.index(gene1)][genes.index(gene2)]
            if sim > best_similarity:
                best_similarity = sim
                best_pair = (gene1, gene2)
    
    return best_pair, best_similarity

def greedy_clustering_parallel(similarity_matrix, genes, max_clusters=2, num_cores=8):
    """
    Greedy clustering algorithm with parallelization.
    """
    clusters = []
    remaining_genes = set(genes)
    
    # counter = 0
    while remaining_genes:
        # Divide the remaining genes into chunks for parallel processing
        chunk_size = len(remaining_genes) // num_cores
        chunks = [list(remaining_genes)[i:i + chunk_size] for i in range(0, len(remaining_genes), chunk_size)]
        
        # Use multiprocessing to find the best pair in parallel
        with mp.Pool(processes=num_cores) as pool:
            results = pool.starmap(
                find_best_pair,
                [(genes, similarity_matrix, chunk) for chunk in chunks]
            )
        
        # Find the overall best pair from the results
        best_pair = None
        best_similarity = -1
        for pair, similarity in results:
            if similarity > best_similarity:
                best_pair = pair
                best_similarity = similarity
        
        if not best_pair:
            break  # No more pairs to add
        
        # Initialize the cluster with the best pair
        cluster = set(best_pair)
        remaining_genes -= cluster
        
        # Add genes to the cluster if they increase average similarity
        for gene in list(remaining_genes):
            candidate_cluster = cluster | {gene}
            new_avg_sim = average_similarity(candidate_cluster, similarity_matrix, genes)
            if new_avg_sim > .45:
                cluster.add(gene)
                remaining_genes.remove(gene)
        if len(cluster) > 2:
            clusters.append(cluster)
        # if "period" in cluster:
                
        # print(counter)
        # counter += 1
        done_gene = []
        for i, cluster in enumerate(clusters):
            for gene_name in cluster:
                if gene_name not in done_gene:
                    done_gene.append(gene_name)
        print(len(clusters),len(cluster), average_similarity(cluster, similarity_matrix, genes), len(done_gene))
            # print(f"Cluster {i}: {cluster}")
        # print(len(clusters), end = " ")
        # if len(clusters) >= max_clusters:
        if "period" in cluster:
            print(f"{cluster}")
            break  # Stop after reaching the maximum number of clusters
    
    return clusters

In [165]:
if __name__ == "__main__":  # Required for multiprocessing on Windows
    # Run the parallel greedy clustering algorithm
    # for i in range(20,55,5):
    
    
    i = 20
    print(i)
    clusters = greedy_clustering_parallel(similarity_matrix, genes, max_clusters=i, num_cores=16)
    print(len(clusters))
# Print the clusters
    for i, cluster in enumerate(clusters):
        if "period" in cluster:
            print(f"Cluster {i}: {cluster}")
        # print(f"Cluster {i}: {cluster}")

20
1 46 0.4512348798498214 46
2 7 0.4507742974142183 53
3 8 0.45943090488088395 61
4 9 0.45259569275732126 70
5 10 0.45036023347860216 80
6 6 0.45030196144269635 86
7 4 0.45053259432233367 90
8 6 0.45706888147493757 96
9 7 0.4592720596115747 103
10 7 0.45217737493621935 110
11 3 0.4584601581739243 113
12 7 0.45032028917645334 120
13 3 0.4557291213464003 123
14 4 0.45273942454911326 127
15 5 0.45348059878117153 132
16 5 0.45512872002056737 137
17 5 0.4526842228916971 142
18 3 0.4604193986441613 145
19 3 0.4527316020467123 148
20 3 0.45146825371226534 151
21 6 0.45551420527414316 157
22 4 0.47302295795338983 161
23 6 0.45307462111287095 167
23 6 0.45307462111287095 167
24 4 0.45061946837395794 171
25 6 0.45039426928018556 177
26 4 0.45431077285540816 181
27 3 0.453413452801517 184
27 3 0.453413452801517 184
28 4 0.4552825987289757 188
29 3 0.45293274884903867 191
30 3 0.45021433873765204 194
31 3 0.45588000565052605 197
32 4 0.46257372948809666 201
32 4 0.46257372948809666 201
33 6 0.457

Process ForkPoolWorker-36430:
Process ForkPoolWorker-36437:
Process ForkPoolWorker-36433:
Process ForkPoolWorker-36441:
Process ForkPoolWorker-36429:
Process ForkPoolWorker-36438:
Process ForkPoolWorker-36435:
Process ForkPoolWorker-36432:
Process ForkPoolWorker-36444:
Process ForkPoolWorker-36439:
Process ForkPoolWorker-36440:
Process ForkPoolWorker-36443:
Process ForkPoolWorker-36431:
Process ForkPoolWorker-36442:
Process ForkPoolWorker-36434:
Process ForkPoolWorker-36436:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call la

In [163]:
for i, cluster in enumerate(clusters):
    # if "period" in cluster:
    #     print(f"Cluster {i}: {cluster}")
    print(f"Cluster {i}: {cluster}, Sim:{average_similarity(cluster, similarity_matrix, genes)}")

Cluster 0: {'DAAM2', 'TAF1', 'LONP1', 'CdsA', 'PI4K2B', 'MCM5', 'OPA1', 'PMCA', 'ARHGAP39', 'ESRRB', 'EZH1', 'TNFAIP8', '4238at7088', 'NOVA1', 'PIK3CB', 'AMOTL1', 'COPA', 'ATP11B', '84at7088', 'GMPS', 'COPE', 'MAP4K1', 'PPP2R2A', 'AP1G1', 'AP1B1', 'NDUFV1', 'HADHB', 'NOP56', 'ATAD1', 'HID1', 'AGAP3', 'Hil', 'MAST4', 'ATP6V1C2', 'BRAF', 'XPO5', 'ATXN2', 'CALU', 'abs', '6335at7088', 'CLCN2', 'MTHFSD', 'drpr', 'PPAT', 'RNF20', 'FLII'}, Sim:0.4512348798498214
Cluster 1: {'DENND5B', 'HECTD1', 'NNT', 'trrap', 'LMNB1', 'RHBDL3', 'ABR'}, Sim:0.4507742974142183
Cluster 2: {'LRSAM1', '1681at7088', 'IL16', 'PLEKHA5', 'kek1', '1892at7088', 'CYP315A1', 'HIVEP1'}, Sim:0.45943090488088395
Cluster 3: {'DNAJC22', 'NOSTRIN', 'hig', 'INTS3', '14318at7088', '8793at7088', 'ATG13', 'BPTF', 'PLCG2'}, Sim:0.45259569275732126
Cluster 4: {'EPN2', 'DARS', 'PDS5A', 'NUBP1', 'PITX2', 'TGFB1I1', 'POLE', 'EIF5B', 'PLS3', 'PIK3R4'}, Sim:0.45036023347860216
Cluster 5: {'13425at7088', 'HCN3', 'Gsc', 'PROP1', 'DUSP22', 

In [103]:
# print(gene_dictionary["period"])
candidate_genes = ["period", "timeless", "cry2", "clock"]
for i, cluster in enumerate(clusters):
    for gene in cluster:
        if gene in candidate_genes:
            print(cluster)

In [ ]:
import matplotlib.pyplot as plt
nx.draw(G, with_labels=True, node_size=50, font_size=8)
plt.show()

In [167]:
!pip3 install matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 6.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 8.7 MB/s eta 0:00:00
